* 진형님 LDA 참고하여 진행
* 동현님 gensim 파일 참고하여 진행

# 1. 데이터 로드

저장된 데이터 가져오기

In [1]:
import pandas as pd
import re
import os

In [2]:
import matplotlib.pyplot as plt

# # 한글 깨짐 방지
# plt.rcParams['font.family'] = "Malgun Gothic"

# # 마이너스 깨짐 방지
# plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 파일 불러오기 
current_dir = os.path.dirname(os.path.abspath('df_labor_real_cleaned3.csv'))
data_dir = os.path.join(current_dir, '..', 'data')
file_path = os.path.join(data_dir, 'df_labor_real_cleaned3.csv')  # 실제 파일명으로 변경

# CSV 파일 읽기
data = pd.read_csv(file_path)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   판례정보일련번호      5016 non-null   int64 
 1   사건번호          5016 non-null   object
 2   선고            5016 non-null   object
 3   판결유형          5016 non-null   object
 4   판시사항          3725 non-null   object
 5   판결요지          3054 non-null   object
 6   참조조문          3724 non-null   object
 7   참조판례          2450 non-null   object
 8   판례내용          5016 non-null   object
 9   판례내용_상단       5016 non-null   object
 10  판례내용_이유       5012 non-null   object
 11  판례내용_이유(전처리)  5016 non-null   object
 12  판례내용_제거(불용어)  5016 non-null   object
 13  판례내용_이유(불용어)  5012 non-null   object
dtypes: int64(1), object(13)
memory usage: 548.8+ KB


In [5]:
df_labor = data['판례내용_이유(불용어)'].copy()
df_labor.info()

<class 'pandas.core.series.Series'>
RangeIndex: 5016 entries, 0 to 5015
Series name: 판례내용_이유(불용어)
Non-Null Count  Dtype 
--------------  ----- 
5012 non-null   object
dtypes: object(1)
memory usage: 39.3+ KB


# 2. 데이터 전처리

## 2-1 토큰화

* 형태소 분석 및 전처리: Okt를 사용하여 한국어 문서에서 명사만을 추출하고, 불용어를 제거합니다. 또한, 한 글자 단어들은 제거하여 의미가 있는 단어들만 남깁니다.
* 단어 사전 및 코퍼스 생성: Dictionary 객체를 사용해 단어 사전을 만들고, 이를 통해 문서 코퍼스를 bag-of-words 형식으로 변환합니다.
* LDA 모델 학습: 주제 수(num_topics)를 설정한 후 LDA 모델을 학습합니다.
주제별 핵심 단어 출력: print_topics 메서드를 사용하여 각 주제의 상위 단어를 출력합니다.

In [13]:
# 토픽수 5개
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 전처리 수행
df_labor.fillna('',inplace =True)  # NaN 값 제거

# CountVectorizer로 단어 빈도 행렬 생성
vectorizer = CountVectorizer(max_df=0.95, min_df=2)  # 불용어 제거 설정
X = vectorizer.fit_transform(df_labor)

# LDA 모델 생성 및 학습
n_topics = 6  # 원하는 토픽 개수
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

# 주제별 상위 단어 출력
terms = vectorizer.get_feature_names_out()
for idx, topic in enumerate(lda.components_):
    print(f"\n주제 {idx + 1}:")
    print([terms[i] for i in topic.argsort()[:-10 - 1:-1]])  # 상위 10개 단어 출력


주제 1:
['지급', '임금', '근로', '수당', '퇴직금', '소득', '금액', '회사', '퇴직', '근로자']

주제 2:
['징계', '회사', '노동조합', '해고', '노조', '근로자', '단체', '노동', '쟁의', '조합원']

주제 3:
['공무원', '임용', '교원', '국가', '학교', '활동', '교육', '사회', '정보', '거부']

주제 4:
['근로자', '사업', '근로', '계약', '고용', '노동조합', '임금', '근무', '업체', '파견']

주제 5:
['회사', '주식회사', '사업', '주식', '거래', '순번', '명의', '법인', '계좌', '자금']

주제 6:
['보험', '재해', '급여', '망인', '사고', '지급', '장해', '손해', '산업', '병원']


In [8]:
import pandas as pd
from konlpy.tag import Okt
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from nltk.corpus import stopwords
import re

# 예시 데이터프레임 준비 (실제 데이터프레임을 여기에 할당하세요)
# 형태소 분석기 초기화
okt = Okt()

# 불용어 설정 (필요시 추가 가능)
# stop_words = set(["이다", "있다", "하다", "되다", "않다", "하다", "보다", "등"])

# # 1. 텍스트 데이터 전처리 및 토큰화
# def preprocess_text(text):
#     # 특수문자 제거 및 소문자 변환
#     text = re.sub(r'[^\w\s]', '', text)
#     tokens = okt.nouns(text)  # 명사 추출
#     return [word for word in tokens if word not in stop_words and len(word) > 1]  # 불용어 및 한 글자 단어 제거

# 전처리 수행
documents = df_labor.dropna().tolist()  # NaN 값 제거
tokenized_docs = [preprocess_text(doc) for doc in documents]

# 2. 단어 사전 및 코퍼스 생성
dictionary = Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(text) for text in tokenized_docs]

# 3. LDA 모델 학습
num_topics = 6  # 주제 개수 설정
lda_model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)

# 4. 주제별 핵심 단어 출력
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

ImportError: cannot import name 'triu' from 'scipy.linalg' (c:\Users\user\anaconda3\envs\project\Lib\site-packages\scipy\linalg\__init__.py)

In [7]:
import pandas as pd
from konlpy.tag import Okt
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import re
import matplotlib.pyplot as plt

# # 예시 데이터프레임 준비 (실제 데이터프레임을 여기에 할당하세요)
# # df_labor = pd.read_csv('your_data.csv')  # 데이터프레임을 불러오는 코드 추가

# # 형태소 분석기 초기화
okt = Okt()

# # 불용어 설정 (필요시 추가 가능)
# stop_words = set(["이다", "있다", "하다", "되다", "않다", "보다", "등"])

# # 1. 텍스트 데이터 전처리 및 토큰화
# def preprocess_text(text):
#     # 특수문자 제거
#     text = re.sub(r'[^\w\s]', '', text)
#     tokens = okt.nouns(text)  # 명사 추출
#     return [word for word in tokens if word not in stop_words and len(word) > 1]  # 불용어 및 한 글자 단어 제거

# 전처리 수행
documents = df_labor.dropna().tolist()  # NaN 값 제거
tokenized_docs = [preprocess_text(doc) for doc in documents]

# 2. 단어 사전 및 코퍼스 생성
dictionary = Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(text) for text in tokenized_docs]

# 3. LDA 모델을 위한 Grid Search 및 Coherence Score 계산
coherence_scores = []
topic_nums = range(2,11)  # 2부터 19까지의 토픽 수

for num_topics in topic_nums:
    lda_model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)
    coherence_model = CoherenceModel(model=lda_model, texts=tokenized_docs, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)

# 4. 결과 시각화
plt.figure(figsize=(10, 5))
plt.plot(topic_nums, coherence_scores, marker='o')
plt.xlabel('Number of Topics')
plt.ylabel('Coherence Score')
plt.title('Coherence Scores for Different Number of Topics')
plt.xticks(topic_nums)
plt.grid()
plt.show()


ImportError: cannot import name 'triu' from 'scipy.linalg' (c:\Users\user\anaconda3\envs\project\Lib\site-packages\scipy\linalg\__init__.py)

In [ ]:
import pandas as pd
from konlpy.tag import Okt  # KoNLPy의 Okt 형태소 분석기를 가져옵니다.
from gensim.corpora.dictionary import Dictionary  # 단어 사전을 만들기 위해 Dictionary를 가져옵니다.
from gensim.models import LdaModel  # LDA 모델을 만들기 위해 LdaModel을 가져옵니다.
from gensim.models.coherencemodel import CoherenceModel  # Coherence Score를 계산하기 위한 모듈을 가져옵니다.
import re  # 정규 표현식을 위한 모듈을 가져옵니다.
import matplotlib.pyplot as plt  # 시각화를 위한 matplotlib.pyplot 모듈을 가져옵니다.

# 예시 데이터프레임 준비 (실제 데이터프레임을 여기에 할당하세요)
# df_labor = pd.read_csv('your_data.csv')  # 데이터프레임을 불러오는 코드 추가

# 형태소 분석기 초기화
okt = Okt()  # Okt 형태소 분석기 객체 생성

# 불용어 설정 (필요시 추가 가능)
stop_words = set(["이다", "있다", "하다", "되다", "않다", "보다", "등"])  # 사용할 불용어 리스트

# 1. 텍스트 데이터 전처리 및 토큰화
def preprocess_text(text):
    # 특수문자 제거: 정규 표현식을 사용하여 특수문자와 기호를 모두 제거합니다.
    text = re.sub(r'[^\w\s]', '', text)
    # 명사 추출: Okt 분석기를 사용하여 텍스트에서 명사만 추출합니다.
    tokens = okt.nouns(text)  
    # 불용어 및 한 글자 단어 제거: 불용어 목록에 없는 단어와 길이가 1보다 큰 단어만 반환합니다.
    return [word for word in tokens if word not in stop_words and len(word) > 1]  

# 전처리 수행
documents = df_labor.dropna().tolist()  # NaN 값 제거 후, 텍스트 데이터를 리스트 형태로 변환
tokenized_docs = [preprocess_text(doc) for doc in documents]  # 각 문서에 대해 전처리 및 토큰화 수행

# 2. 단어 사전 및 코퍼스 생성
dictionary = Dictionary(tokenized_docs)  # 전처리된 문서로부터 단어 사전을 생성합니다.
corpus = [dictionary.doc2bow(text) for text in tokenized_docs]  # 각 문서의 단어를 ID와 빈도로 변환한 코퍼스를 생성합니다.

# 3. LDA 모델을 위한 Grid Search 및 Coherence Score 계산
coherence_scores = []  # Coherence Score를 저장할 리스트 초기화
topic_nums = range(2, 20)  # 2부터 19까지의 토픽 수 설정 (최적의 토픽 수를 찾기 위한 범위)

for num_topics in topic_nums:  # 설정한 각 토픽 수에 대해 반복
    # LDA 모델 학습: 지정된 토픽 수로 LDA 모델을 학습합니다.
    lda_model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)
    
    # Coherence Score 계산: LDA 모델의 일관성을 측정합니다.
    coherence_model = CoherenceModel(model=lda_model, texts=tokenized_docs, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()  # Coherence Score를 계산합니다.
    coherence_scores.append(coherence_score)  # 계산된 Coherence Score를 리스트에 추가합니다.

# 4. 결과 시각화
plt.figure(figsize=(10, 5))  # 플롯 크기 설정
plt.plot(topic_nums, coherence_scores, marker='o')  # 각 토픽 수에 따른 Coherence Score를 플롯에 추가
plt.xlabel('Number of Topics')  # x축 레이블
plt.ylabel('Coherence Score')  # y축 레이블
plt.title('Coherence Scores for Different Number of Topics')  # 플롯 제목
plt.xticks(topic_nums)  # x축 눈금을 설정
plt.grid()  # 그리드 표시
plt.show()  # 플롯을 화면에 표시


In [ ]:
import gensim
print (gensim.__version__)

4.3.3


In [ ]:
from gensim.summarization import summarize

summary = summarize(df['판례내용'].str.cat(sep=' '), ratio=0.1)  # 10% 비율로 요약
print(summary)